In [1]:
pip install tavily-python requests beautifulsoup4 transformers torch

Note: you may need to restart the kernel to use updated packages.


In [26]:
from tavily import TavilyClient
import requests
from bs4 import BeautifulSoup
def give_content(query="delhi , artifical intelligence"):
    client = TavilyClient(api_key="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")
    response = client.search(
        query=query,
        search_depth="advanced",
        topic="news",
        time_range="year",
        include_images=True
    )
    for i in response['results']:

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }
        URL = i['url']
        response = requests.get(URL, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
        
            # Example: Extract all paragraph text
            paragraphs = [p.text.strip() for p in soup.find_all('p')]
            
            # Print extracted text
            return paragraphs
    print("Failed to retrieve the webpage")

In [28]:
txt = give_content()

In [36]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from typing import List, Tuple

class TextSummarizer:
    def __init__(self, model_name: str = "t5-small"):
        """
        Initialize the summarizer with T5 model and tokenizer
        Args:
            model_name: Name of the T5 model to use (default: 't5-small')
        """
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name).to(self.device)

    def preprocess_text(self, paragraphs: List[str], task: str) -> str:
        """
        Combine paragraphs and add prefix for summarization or title generation
        Args:
            paragraphs: List of paragraph strings
            task: Task type ('summarize' or 'title')
        Returns:
            Preprocessed text ready for processing
        """
        combined_text = "\n".join(paragraphs)
        return f"{task}: {combined_text}"

    def generate_text(self, input_text: str, max_length: int) -> str:
        """
        Generate text using T5 model
        Args:
            input_text: Input text for the model
            max_length: Maximum length of the generated text
        Returns:
            Generated text
        """
        inputs = self.tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(self.device)

        output_ids = self.model.generate(
            inputs,
            max_length=max_length,
            num_beams=5,
            length_penalty=2.0,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

        return self.tokenizer.decode(output_ids[0], skip_special_tokens=True)

    def summarize_and_title(self, paragraphs: List[str]) -> Tuple[str, str]:
        """
        Generate both a summary and a title
        Args:
            paragraphs: List of paragraph strings
        Returns:
            Tuple of (summary, title)
        """
        summary = self.generate_text(self.preprocess_text(paragraphs, "summarize"), max_length=150)
        title = self.generate_text(self.preprocess_text(paragraphs, "title"), max_length=20)
        return summary, title

# Initialize summarizer
summarizer = TextSummarizer()

# Example usage
summary, title = summarizer.summarize_and_title(txt)

Title: .  2025 Hyderabad Media House Limited/The Hans India. All rights reserved.
Summary: the coEs will be led by top educational institutions, in consortium with industry partners and startups. they will democratise AI innovations and research to revolutionize healthcare delivery, strengthen food security and address critical urban challenges. the interdisciplinary research, develop cutting-edge applications and create scalable solutions in these three areas.


In [38]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Define OAuth 2.0 scope
SCOPES = ["https://www.googleapis.com/auth/blogger"]

def authenticate():
    """Authenticate and get OAuth 2.0 credentials."""
    flow = InstalledAppFlow.from_client_secrets_file(
        "credentials.json", SCOPES
    )
    creds = flow.run_local_server(port=0)
    return creds

def create_blog_post(creds,title,content):
    """Create a new blog post using Blogger API."""
    service = build("blogger", "v3", credentials=creds)

    blog_id = "7894317275629685509"  # Replace with your actual Blog ID
    post_body = {
        "title": title,
        "content": content
    }

    post = service.posts().insert(blogId=blog_id, body=post_body).execute()
    print("Post created:", post["url"])
    return spost["url"]

In [42]:
# Authenticate and create a post
creds = authenticate()
link_to_website = create_blog_post(creds,title,summary)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=183208013542-6kfpqdql03aoljkv0ehbemj69h6jp1vn.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57105%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fblogger&state=MUbjdoVDNrZ3QIoYmH2G9BiTqdqmcy&access_type=offline
Post created: http://newsflipr.blogspot.com/2025/02/2025-hyderabad-media-house-limitedthe.html


In [46]:
print(link_to_website)

None
